In [7]:
import tweepy
import re

In [8]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

In [9]:
pd.options.display.max_colwidth = 1000


In [10]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [11]:
input_hashtag = "giletjaune"
n_tweets = 1
language = "fr"
since_date = "2018-03-07"

In [7]:
####request to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify = True)

inc = 0
for tweet in tweepy.Cursor(api.search,q="#"+input_hashtag,#count=n_tweets,
                           lang=language,
                           since=since_date, tweet_mode='extended').items():
    inc = +1
    with open('tweets/tweet'+str(tweet.id)+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets' folder
        
        tweet_json = tweet._json
        json.dump(tweet_json, file)
        

Rate limit reached. Sleeping for: 381


KeyboardInterrupt: 

In [12]:
def is_a_retweet(tweet):
    #### returns a boolean 
    return tweet.get('retweeted_status',None) != None

In [13]:
#initialisation of databases, df files

d_users = {'user_id': [],
           'name':[],
           'screen_name':[],
           'location':[],
           'description':[],
           'url':[],
           'followers_count':[],
           'friends_count':[],
           'listed_count':[],
           'created_at':[],
           'favourites_count':[],
           'geo_enabled':[],
           'verified':[],
           'statuses_count':[],
           'lang':[],
           'contributors_enabled':[]
          }
df_users = pd.DataFrame(data=d_users)

d_tweets = {'tweet_id': [],
            'created_at': [],
            'text': [],
            'truncated': [],
            'source': [],
            'in_reply_to_status_id': [],
            'in_reply_to_user_id': [],
            'in_reply_to_screen_name': [],
            'user_id': [],
            'geo': [],
            'coordinates': [],
            'place': [],
            'contributors': [],
            'is_quote_status': [],
            'retweet_count': [],
            'favorite_count': [],
            'favorited': [],
            'retweeted': [],
            'lang': []         
           }
df_tweets = pd.DataFrame(data=d_tweets)


d_hashtags={'hashtag': [], 'tweet_id': []}
df_hashtags = pd.DataFrame(data=d_hashtags)

d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
df_users_mentions = pd.DataFrame(data=d_users_mentions)

d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
df_retweet_users= pd.DataFrame(data=d_retweet_users)

d_influent_users = {'user_id':[], 
                    'name':[],
                    'retweet_count':[],
                    'favorite_count':[], 
                    'description':[] 
                   }

df_influent_users = pd.DataFrame(data=d_influent_users)



In [14]:
def store_hashtag(tweet):
    global input_hashtag
    global df_hashtags
    if tweet['entities']['hashtags']==[]:
        df_hashtags = df_hashtags.append({'hashtag': input_hashtag ,'tweet_id':tweet['id_str']}, ignore_index=True)
    else:
        for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

def store_users(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

def store_tweet(tweet):
    global df_tweets
    text = re.sub(r"http\S+", "", tweet['full_text'])
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': text,
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)
        
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
        
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        

In [15]:
#### creation of the databases

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os
path_to_json = 'tweets/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open("tweets/"+json_file) as tweet:
        tweet = json.load(tweet)
        if is_a_retweet(tweet):
            store_retweet_user(tweet)
        else:
            store_tweet(tweet)
            store_users(tweet)
            store_hashtag(tweet)
            store_user_mentions(tweet)
        
        
        

In [109]:
df_tweets.head(10);


In [106]:
df_tweets['text'].iloc[0]

'Insupportable d\'entendre le #GiletJaune en roue libre sur #LCI dire, par exemple, que l\'élection d\'#EmmanuelMacron est "une erreur de l\'Histoire", et ajouter "j\'aimerais bien qu\'il bosse au lieu de faire du ski". Sans que la journaliste ne le pousse à s\'expliquer davantage...'

In [110]:
#primary keys are ok?
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())


False
False


# Aggregations


## Top users

In [111]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)



In [112]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [113]:
df_most_retweeted_users_with_info.head(15);

In [114]:
df_most_favorited_users_with_info.head(15);

In [115]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [116]:
print(df_most_retweeted_users_with_info['name'])
print(df_most_favorited_users_with_info['name'])

user_id
472852289                        Brèves de presse
388972391                            Anasse Kazib
338955445                     de Cabarrus Thierry
2148731538                    Jean-Yves Le Gallou
742382030                         Gaëtan #FBPE  🌍
2470583048                         Charles Baudry
2653885316                         jean garrigues
203894274                         Cécile Ollivier
1028668294751039488            LesGiletsJaunes.fr
3688356795                     Jean-Pierre Bordes
840670437050875904             BleuBlancRouge4369
804606773848133633              Barthe Philippe 🐝
817189792769339392       Abbé Rézina 🇫🇷 ن 2⃣3⃣⛽️🚕
996698079180050433     ⚡️P'tweet Revue de Presse🗞
1064054889452634112      🇫🇷⭐️GiletsJauneRouen⭐️🇫🇷
Name: name, dtype: object
user_id
472852289                 Brèves de presse
338955445              de Cabarrus Thierry
388972391                     Anasse Kazib
742382030                  Gaëtan #FBPE  🌍
2653885316                  jean gar

In [117]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [118]:
df_most_favorited_users_with_info;

In [119]:
top_influencers_fav = df_most_favorited_users_with_info[['screen_name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['screen_name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);
top_influencers_fav['Number of followers'] = top_influencers_fav['Number of followers'].astype(int)
top_influencers_fav['Number of friends'] = top_influencers_fav['Number of friends'].astype(int)
top_influencers_rt['Number of followers'] = top_influencers_rt['Number of followers'].astype(int)
top_influencers_rt['Number of friends'] = top_influencers_rt['Number of friends'].astype(int)

In [120]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

In [123]:
top_influencers_fav_names = top_influencers_fav['Name'].values
top_influencers_rt_names = top_influencers_rt['Name'].values
print(top_influencers_rt_names)

['Brevesdepresse' 'AnasseKazib' 'tcabarrus' 'jylgallou' 'gaetan34070'
 'CharlesBaudry' 'jeangarr78' 'cecileollivier' 'GiletsJaunesFr'
 'jpbordes4873' 'BBR4369' 'philippe_barthe' 'Abbe_Rezina' 'P_tweet_presse'
 'dms768']


## Top hashtags for this input hashtags and by users

In [124]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [125]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [126]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [127]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['name','hashtag']).size().reset_index()

In [128]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;


In [129]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['name'] == 'LesGiletsJaunes.fr']
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

def get_related_hashtags_by_username(name):
    d_name = df_aggretated[df_aggretated['Name'] == name][['#',"Used"]].sort_values("Used", ascending = False)
    return d_name

In [130]:
get_tweet_by_username('LesGiletsJaunes.fr')


,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
1028668294751039488,1107771021665492998,Vous demandez une augmentation de 200 balles par mois ?\nPas de problème ...\n#GiletJaune #GiletsJaunes #YellowVests #SMIC \n#assembleenationale #senat #Europeennes2019 \n#droitsdelhomme #Amnestyinternational #ViolencesPolicieres #ONU\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews,Mon Mar 18 22:29:27 +0000 2019,52.0,47.0
1028668294751039488,1107783190708658176,"Agence BFM ( Benalla Fouille Mer ... ) :\nUn bandeau sur l’œil , comme Jean-Marie Le Pen ! ... le bras tendu !\nL'appartenance à la mouvance néo-extrême ne fait plus aucun doute !\nIls sont tous complices !\n#GiletJaune #GiletsJaunes #facknews d'état !",Mon Mar 18 23:17:48 +0000 2019,43.0,17.0
1028668294751039488,1107790134890676225,"Didier Maïsto : Retour sur l’Acte 18, décisions du Premier ministre et intervention dans ce live de Bruno Pomart, ancien instructeur du RAID.\n\n#GiletJaune #GiletsJaunes #assembleenationale #senat #Europeennes2019 \n#droitsdelhomme #ViolencesPolicieres #ONU",Mon Mar 18 23:45:24 +0000 2019,7.0,8.0


In [131]:
#output 
for name in top_influencers_rt_names:
    df_rel = get_related_hashtags_by_username(name)
    print(name)
    df_rel.to_csv('top_hashtags_by_'+name+'.csv',index=False)
    df_rel=[]

Brevesdepresse
AnasseKazib
tcabarrus
jylgallou
gaetan34070
CharlesBaudry
jeangarr78
cecileollivier
GiletsJaunesFr
jpbordes4873
BBR4369
philippe_barthe
Abbe_Rezina
P_tweet_presse
dms768


In [132]:
df_tweets['created_at'].values;


# NLP


In [5]:
import spacy


In [3]:
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import fr_core_news_sm
nlp = fr_core_news_sm.load()
#!python -m spacy download fr_core_web_lg

In [19]:
doc = nlp(df_tweets["text"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [22]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [23]:
review = str(" ".join([i.lemma_ for i in doc]))

In [24]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [25]:
for i in nlp(review):
    print(i,"=>",i.pos_)

rien => PRON
que => ADV
sur => ADP
le => DET
journée => NOUN
de => ADP
samedi => NOUN
dernier => ADJ
, => PUNCT
le => DET
ville => NOUN
de => ADP
Paris => PROPN
estimer => VERB
le => DET
dégât => NOUN
à => ADP
5 => NUM
million => NOUN
d’ => PUNCT
euro => NOUN
. => PUNCT
le => DET
facture => NOUN
# => ADJ
giletjaun => ADJ
depuis => ADP
le => DET
début => NOUN
du => DET
mouvement => NOUN
en => ADP
être => AUX
désormais => ADV
à => ADP
20 => NUM
million => NOUN
. => PUNCT
@rtlfrance => NOUN


In [26]:
parser = French()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [27]:
tqdm.pandas()
df_tweets["processed_text"] = df_tweets["text"].progress_apply(spacy_tokenizer)

100%|██████████| 427/427 [00:00<00:00, 449.83it/s]


In [31]:
df_tweets.head(10)["processed_text"];

In [33]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words=stopwords, lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df_tweets["processed_text"])

In [34]:
##How many topics do you want to find??
NUM_TOPICS = 6

In [35]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [37]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [38]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [39]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [40]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('giletsjaunes', 69.21013725551052), ('acte', 42.461490950963885), ('gilet', 40.08763852990299), ('jaune', 33.124712979441455), ('macron', 29.684920407783636), ('mars', 25.983699005499744), ('france', 23.74751311300433), ('manifestation', 18.476011889055396), ('ric', 14.726885081686591), ('actexix', 13.85690810087591)]
Topic 1:
[('paris', 59.000335553069476), ('giletsjaunes', 57.69304262721306), ('dan', 45.224758723198555), ('faire', 34.97325448448134), ('acte', 29.328155168376384), ('voir', 27.009388073566342), ('place', 26.79715670089685), ('giletsjaunesparis', 25.49789456923383), ('gilets', 25.38703311777713), ('jaunes', 24.002659423204715)]
Topic 2:
[('plaire', 26.82543093737326), ('ais', 16.620569283298394), ('fran', 15.612052029307149), ('bon', 14.622490915189907), ('peupler', 11.575850675497612), ('giletsjaunes', 11.183301898820886), ('gouvernement', 10.189968514066042), ('faire', 7.640129749905296), ('emmanuelmacron', 7.5315749577708475), ('entendre', 6.970

In [42]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('giletsjaunes', 2.756361585382929), ('acte', 2.4316880569272037), ('macron', 1.4384458626710162), ('france', 0.9537181928525174), ('actexviii', 0.7369346431442704), ('europeennes', 0.5994131727076925), ('violence', 0.5395062898715997), ('castaner', 0.4461087459990488), ('champselysees', 0.4458829271298633), ('manifestation', 0.44169869841918613)]
Topic 1:
[('gilet', 2.334951652678174), ('jaune', 1.7344137434082019), ('giletjaunes', 0.6906740677680913), ('jaunes', 0.6671580529147063), ('giletsjaunes', 0.6365629665387219), ('mars', 0.457974578393437), ('champs', 0.20148482040890456), ('appel', 0.19357755236984397), ('samedi', 0.18945703653138327), ('elys', 0.16407420359489702)]
Topic 2:
[('faire', 1.7234702369709083), ('plaire', 1.2073678301931465), ('aller', 0.8586879141208608), ('voir', 0.8579599357893832), ('mettre', 0.7005328829927717), ('pouvoir', 0.6315897896635945), ('devoir', 0.5567553318249884), ('dan', 0.4817922058304429), ('falloir', 0.4815712331564718), 

In [43]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('giletsjaunes', 0.6346853331573089), ('acte', 0.3388326778809631), ('macron', 0.20598006386094234), ('gilet', 0.17806976523662074), ('france', 0.1704697031485287), ('faire', 0.16429540848461502), ('paris', 0.16134444537803552), ('dan', 0.1571576528696538), ('jaune', 0.15235303248876408), ('jaunes', 0.11479174486940306)]
Topic 1:
[('gilet', 0.6511102062788277), ('jaune', 0.5149969707175744), ('jaunes', 0.1920759185362585), ('giletjaunes', 0.1705008039279021), ('mars', 0.09036270738657123), ('samedi', 0.06866693299425469), ('champs', 0.06112270333678422), ('aller', 0.060110660483243865), ('elys', 0.05251646924065648), ('appel', 0.04914386926688137)]
Topic 2:
[('giletsjaunes', 0.28654615144135415), ('gilet', 0.16324479942380846), ('mois', 0.1142376645461183), ('claration', 0.1128008323211415), ('excessivement', 0.10948249340473494), ('ena', 0.10948249340473494), ('dialogue', 0.10948249340473494), ('cout', 0.10948249340473494), ('compromettre', 0.10948249340473494), (

In [44]:
# Transforming an individual sentence
text = spacy_tokenizer("Les gilets jaune tous unis contre Macron. Tous dans la rue jusqu'à la démission")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.88046869 0.02405957 0.02386698 0.02382779 0.02384151 0.02393547]


In [45]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1       6.351091  -6.803735       1        1  25.807636
0      52.821205 -36.725849       2        1  25.613987
3      49.167465  28.147064       3        1  16.433988
2     -13.658890  44.716759       4        1  12.142427
5      -7.747123 -60.244652       5        1  11.662537
4     -48.831783  -9.912099       6        1   8.339424, topic_info=    Category        Freq               Term       Total  loglift  logprob
8    Default   38.000000              aller   38.000000  30.0000  30.0000
159  Default   41.000000             plaire   41.000000  29.0000  29.0000
170  Default   25.000000            prendre   25.000000  28.0000  28.0000
59   Default   26.000000             drouet   26.000000  27.0000  27.0000
145  Default   43.000000              paris   43.000000  26.0000  26.0000
91   Default   40.000000              gilet   40.000000  25.0000  25.0000
167  Default   17.000000             porter   17.000000  24.0000  24.0000
7    Default   17.000000                ais   17.000000  23.0000  23.0000
68   Default   20.000000        ephilippepm   20.000000  22.0000  22.0000
2    Default   63.000000               acte   63.000000  21.0000  21.0000
86   Default   19.000000               fran   19.000000  20.0000  20.0000
25   Default   16.000000                bon   16.000000  19.0000  19.0000
133  Default   26.000000               mars   26.000000  18.0000  18.0000
134  Default   23.000000             mettre   23.000000  17.0000  17.0000
50   Default   58.000000                dan   58.000000  16.0000  16.0000
178  Default   31.000000                ric   31.000000  15.0000  15.0000
126  Default   42.000000             macron   42.000000  14.0000  14.0000
113  Default   45.000000              jaune   45.000000  13.0000  13.0000
137  Default   14.000000               mois   14.000000  12.0000  12.0000
56   Default   14.000000           domicile   14.000000  11.0000  11.0000
74   Default   55.000000              faire   55.000000  10.0000  10.0000
155  Default   14.000000            peupler   14.000000   9.0000   9.0000
33   Default   16.000000             champs   16.000000   8.0000   8.0000
130  Default   33.000000      manifestation   33.000000   7.0000   7.0000
156  Default   19.000000              place   19.000000   6.0000   6.0000
17   Default    8.000000           attacher    8.000000   5.0000   5.0000
95   Default  144.000000       giletsjaunes  144.000000   4.0000   4.0000
96   Default   18.000000  giletsjaunesparis   18.000000   3.0000   3.0000
93   Default   18.000000             gilets   18.000000   2.0000   2.0000
69   Default   12.000000               eric   12.000000   1.0000   1.0000
..       ...         ...                ...         ...      ...      ...
143   Topic6    4.660794              opter    5.493584   2.3198  -3.8769
23    Topic6    4.660481              bient    5.494721   2.3195  -3.8770
64    Topic6    4.659031                ena    5.493172   2.3195  -3.8773
54    Topic6    4.661205           dialogue    5.496884   2.3193  -3.8768
165   Topic6    4.658744      polytechnique    5.495035   2.3191  -3.8773
46    Topic6    4.663500               cout    5.501761   2.3189  -3.8763
73    Topic6    4.665260      excessivement    5.504498   2.3188  -3.8759
179   Topic6    4.662290         rieusement    5.501292   2.3187  -3.8766
19    Topic6    4.662434             ballot    5.502050   2.3186  -3.8766
41    Topic6    4.660599       compromettre    5.500802   2.3184  -3.8769
149   Topic6    6.948197              payer    8.576705   2.2736  -3.4776
194   Topic6    5.588637            suffire    7.222455   2.2277  -3.6954
170   Topic6   17.371777            prendre   25.105866   2.1159  -2.5612
103   Topic6    3.608684              haine    5.443886   2.0730  -4.1327
137   Topic6    9.338440               mois   14.497784   2.0443  -3.1820
68    Topic6   11.976523       